In [22]:
import pandas as pd
import math

In [39]:
def getdataset(filename):
    dataframe = pd.read_csv(filename)
    
    num = {" US.":0, " Europe.":1, " Japan.": 2}
    dataframe = dataframe.replace(num)
    
    dataset = []
    for index, transaction in dataframe.iterrows():
        row = []
        for item in transaction:
            item = int(item)
            row.append(item)
        dataset.append(row)
            
    return dataset

In [40]:
def testTrainSplit(dataset, ratio):
    split = int(ratio*len(dataset))
    
    train = dataset[:split]
    test = dataset[split:]
    
    return train, test

In [41]:
def getmeanSD(item):
    mean = sum(item)/float(len(item))
    variance = 0
    
    for x in item:
        variance = variance + (x-mean)**2
    variance = variance/(len(item)-1)

    return mean, math.sqrt(variance)

In [51]:
def getSummary(dataset):
    
    diction = {}
    for row in dataset:
        if row[-1] not in diction:
            diction[row[-1]] = []
        diction[row[-1]].append(row)
        
    summary = {}
    for key in diction:
        summary[key] = []
        for item in tuple(zip(*(diction[key]))):
            mean, sd = getmeanSD(item)
            summary[key].append((mean, sd))
        del(summary[key][-1])
    return summary

In [66]:
def calcProb(x, mean, sd):
    e = math.exp( -((x-mean)**2))/(2*(sd**2) )
    prob = (1/(math.sqrt(2*math.pi)*sd))* e
    return prob

In [70]:
def classify(data, row):
    
    probabilities = {}
    for key in data:
        probabilities[key] = 1
        i = 0
        for item in data[key]:
            probabilities[key] *= calcProb(row[i], item[0], item[1])
            i += 1
    highestprob = 0
    label = 0
    for key in probabilities:
        if probabilities[key] > highestprob:
            highestprob = probabilities[key]
            label = key
    return label

In [71]:
def getPredict(data, test):
    prediction = []
    
    for row in test:
        classLabel = classify(data, row)
        prediction.append(classLabel)
    
    accuracy = 0   
    for row in test:
        if row[-1] == classLabel:
            accuracy += 1
    return(accuracy/len(test))

In [73]:
filename = "cars.csv"
ds = getdataset(filename)
train, test = testTrainSplit(ds, 0.8)
dataSummarized = getSummary(train) 
print(dataSummarized)
testResult = getPredict(dataSummarized, test)
print(testResult)

{0: [(19.484375, 6.136228258799903), (6.453125, 1.6307062138550623), (257.9453125, 102.86435595426545), (122.3359375, 41.8356885931026), (3435.171875, 779.8370626240625), (14.9140625, 2.918923714249192), (1976.453125, 3.645425080254432)], 1: [(27.025641025641026, 6.045821434272988), (4.102564102564102, 0.38353424295565597), (107.05128205128206, 21.596720898697757), (78.64102564102564, 19.739737079632278), (2391.3076923076924, 456.24480696950366), (16.82051282051282, 3.2107696640147867), (1976.128205128205, 2.903532215291604)], 2: [(29.853658536585368, 6.142316239049223), (4.097560975609756, 0.583304296606003), (103.85365853658537, 23.466530395021927), (79.65853658536585, 18.548058868918815), (2235.170731707317, 322.2622303682999), (16.463414634146343, 1.8041280577554601), (1978.1951219512196, 3.551193547211428)]}
0.6415094339622641
